In [0]:
#Mounting the google drive to access the image folder
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#Importing the necessary libraries
import numpy as np 
import pandas as pd 
import os

In [0]:
from keras import layers
from keras import models
from keras.utils import to_categorical
import matplotlib.pyplot as plt
from os import listdir, makedirs
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, ResNet50, VGG19, InceptionV3
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras import optimizers, regularizers
from keras.optimizers import SGD
from glob import glob
import cv2
import warnings 
warnings.filterwarnings('ignore')

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.losses import categorical_crossentropy
from keras.models import Model
from keras.models import load_model
from keras.preprocessing import image
from mpl_toolkits.axes_grid1 import ImageGrid

In [0]:
import tensorflow as tf
import time

In [0]:
#Saving the resized image directory to variable
root_directory ='/content/drive/My Drive/Resized_images'

In [0]:
train_directory = os.path.join(root_directory, 'train')

In [0]:
#Finding out the number of images for each classes
classifications = os.listdir(train_directory)
for classes in classifications:
    if classes != '.DS_Store':
        print('{}: {} images'.format(classes, len(os.listdir(os.path.join(train_directory, classes)))))

c0: 2489 images
c1: 2269 images
c2: 2325 images
c3: 2346 images
c4: 2334 images
c5: 2312 images
c6: 2325 images
c7: 2002 images
c8: 1911 images
c9: 2139 images


In [0]:
#Creating a dataset to easily access the images
class_list =  ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6','c7', 'c8', 'c9']
train = []
for classKey, classValue in enumerate(class_list):
    for file in os.listdir(os.path.join(train_directory, classValue)):
        train.append(['train/{}/{}'.format(classValue, file), classKey, classValue])
        
training_data = pd.DataFrame(train, columns=['image', 'class_key', 'class_value'])
training_data.head(10)

,image,class_key,class_value
0,train/c0/img_39359.jpg,0,c0
1,train/c0/img_66991.jpg,0,c0
2,train/c0/img_84384.jpg,0,c0
3,train/c0/img_77136.jpg,0,c0
4,train/c0/img_97165.jpg,0,c0
5,train/c0/img_45276.jpg,0,c0
6,train/c0/img_28357.jpg,0,c0
7,train/c0/img_59145.jpg,0,c0
8,train/c0/img_63859.jpg,0,c0
9,train/c0/img_90333.jpg,0,c0


In [0]:
ensemble_resize0 = training_data[training_data.class_value == 'c0'].head(100)

In [0]:
ensemble_resize1 = training_data[training_data.class_value == 'c1'].head(100)

In [0]:
ensemble_resize2 = training_data[training_data.class_value == 'c2'].head(100)

In [0]:
ensemble_resize3 = training_data[training_data.class_value == 'c3'].head(100)

In [0]:
ensemble_resize4 = training_data[training_data.class_value == 'c4'].head(100)

In [0]:
ensemble_resize5 = training_data[training_data.class_value == 'c5'].head(100)

In [0]:
ensemble_resize6 = training_data[training_data.class_value == 'c6'].head(100)

In [0]:
ensemble_resize7 = training_data[training_data.class_value == 'c7'].head(100)

In [0]:
ensemble_resize8 = training_data[training_data.class_value == 'c8'].head(100)

In [0]:
ensemble_resize9 = training_data[training_data.class_value == 'c9'].head(100)

In [0]:
#Creating a samller dataset that contains only 100 images from each class
ensemble_resize = pd.concat([ensemble_resize0,ensemble_resize1,ensemble_resize2,ensemble_resize3,ensemble_resize4,ensemble_resize5,ensemble_resize6,ensemble_resize7,ensemble_resize8,ensemble_resize9], ignore_index=True)

In [0]:
#Saving those 1000 images to a new directory
import cv2
from keras.preprocessing import image
def save_images(image_location):
    original_image = cv2.imread(os.path.join(root_directory, image_location))
    path = '/content/drive/My Drive/Ensemble_resize'
    cv2.imwrite(os.path.join(path , image_location), original_image)

In [0]:
dim_inp = 128

for i, file in enumerate(ensemble_resize['image']):
    save_images(file);
    print("Training Image saved")

Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Image saved
Training Imag

In [0]:
#Finding the number of images for each class in the new directory
classifications = os.listdir(train_directory)
for classes in classifications:
    if classes != '.DS_Store':
        print('{}: {} images'.format(classes, len(os.listdir(os.path.join('/content/drive/My Drive/Ensemble_resize/train',classes)))))

c0: 100 images
c1: 100 images
c2: 100 images
c3: 100 images
c4: 100 images
c5: 100 images
c6: 100 images
c7: 100 images
c8: 100 images
c9: 100 images


In [0]:
#Finding out the number of output classes
class_numbers = len(np.unique(ensemble_resize["class_key"]))
print("number of classes in the given images: ", class_numbers)

number of classes in the given images:  10


In [0]:
#Performing Test Train Split
from sklearn.model_selection  import train_test_split
output = ensemble_resize["class_key"]

test_size = 0.30
seed = 10
X_train, X_val, y_train, y_val = train_test_split(ensemble_resize, output, test_size=test_size, random_state=seed)

In [0]:
print("Dimensions of the training dataset: {}".format(X_train.shape))
print("Dimensions of the validation dataset: {}".format(X_val.shape))

Dimensions of the training dataset: (700, 3)
Dimensions of the validation dataset: (300, 3)


In [0]:
ensemble_directory='/content/drive/My Drive/Ensemble_resize'

In [0]:
#Finding out the image dimension for train and validation data
import cv2
from keras.preprocessing import image
def dimension_images(image_location, size):
    original_image = cv2.imread(os.path.join(ensemble_directory, image_location))
    image_dimension = image.img_to_array(original_image)
    image_dimension = np.expand_dims(image_dimension.copy(), axis=0)
    return image_dimension

In [0]:
dim_inp = 128

# reading image file for traing dataset
X_train_parameters = np.zeros((len(X_train), dim_inp, dim_inp, 3), dtype='float32')
for i, file in enumerate(X_train['image']):
    data_image = dimension_images(file, (dim_inp, dim_inp))
    X_train_parameters[i] = data_image
print('Training Images shape: {} size: {:,}'.format(X_train_parameters.shape, X_train_parameters.size))

Training Images shape: (700, 128, 128, 3) size: 34,406,400


In [0]:
X_val_parameters = np.zeros((len(X_val), dim_inp, dim_inp, 3), dtype='float32')
for i, file in enumerate(X_val['image']):
    data_image = dimension_images(file, (dim_inp, dim_inp))
    X_val_parameters[i] = data_image
print('Validation Images shape: {} size: {:,}'.format(X_val_parameters.shape, X_val_parameters.size))

Validation Images shape: (300, 128, 128, 3) size: 14,745,600


In [0]:
X_train_parameters = X_train_parameters.astype('float32')/255
X_val_parameters = X_val_parameters.astype('float32')/255

In [0]:
y_train_category = keras.utils.to_categorical(y_train, num_classes=class_numbers)
y_val_category = keras.utils.to_categorical(y_val, num_classes=class_numbers)

In [0]:
print("Training data shape:")
print("Features: ",X_train_parameters.shape)
print("Target: ",y_train_category.shape)

Training data shape:
Features:  (700, 128, 128, 3)
Target:  (700, 10)


In [0]:
print("\nValidation data shape:")
print("Features: ",X_val_parameters.shape)
print("Target: ",y_val_category.shape)


Validation data shape:
Features:  (300, 128, 128, 3)
Target:  (300, 10)


In [0]:
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os

In [0]:
in_parameters = X_train_parameters[0,:,:,:].shape
mod_parameters = Input(shape=in_parameters)

In [0]:
#First Model - ConvPool-CNN
def cnn_convolution_pooling(model_input: Tensor) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='cnn_convolution_pooling')
    
    return model

In [0]:
model1 = cnn_convolution_pooling(mod_parameters)

In [0]:
def build_model(model: training.Model, num_epochs: int) -> Tuple [History, str]: 
    
    model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc']) 
    filepath = '/content/drive/My Drive/weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True,
                                                 save_best_only=True, mode='auto', period=1)
    tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
    history = model.fit(x=X_train_parameters, y=y_train_category, batch_size=32, 
                     epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
    weight_files = glob.glob('/content/drive/My Drive/weights/*')
    weight_file = max(weight_files, key=os.path.getctime)
    return history, weight_file

In [0]:
#ConvPool-CNN Output
number_of_epochs= 5
model1_history, model1_weights = build_model(model1, number_of_epochs)

Train on 560 samples, validate on 140 samples
Epoch 1/5
560/560 [==============================] - 470s 839ms/step - loss: 2.3078 - acc: 0.0911 - val_loss: 2.3021 - val_acc: 0.1214
Epoch 2/5
560/560 [==============================] - 470s 838ms/step - loss: 2.3027 - acc: 0.1054 - val_loss: 2.3026 - val_acc: 0.0857
Epoch 3/5
560/560 [==============================] - 470s 839ms/step - loss: 2.3026 - acc: 0.1107 - val_loss: 2.3017 - val_acc: 0.1214
Epoch 4/5
560/560 [==============================] - 469s 837ms/step - loss: 2.3025 - acc: 0.0929 - val_loss: 2.3023 - val_acc: 0.0857
Epoch 5/5
560/560 [==============================] - 475s 849ms/step - loss: 2.3024 - acc: 0.1071 - val_loss: 2.3021 - val_acc: 0.1214


In [0]:
#Second Model - All-CNN

def cnn_2(model_input: Tensor) -> training.Model:
    
    x = Conv2D(96, kernel_size=(3, 3), activation='relu', padding = 'same')(model_input)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2)(x)
    x = Conv2D(192, (3, 3), activation='relu', padding = 'same')(x)
    x = Conv2D(192, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
        
    model = Model(model_input, x, name='cnn_2')
    
    return model

In [0]:
model_2 = cnn_2(mod_parameters)

In [0]:
#All-CNN output
model2_history, model2_weights = build_model(model_2, number_of_epochs)

Train on 560 samples, validate on 140 samples
Epoch 1/5
560/560 [==============================] - 331s 591ms/step - loss: 2.3080 - acc: 0.0750 - val_loss: 2.3026 - val_acc: 0.1214
Epoch 2/5
560/560 [==============================] - 346s 618ms/step - loss: 2.3028 - acc: 0.1071 - val_loss: 2.3021 - val_acc: 0.1214
Epoch 3/5
560/560 [==============================] - 341s 609ms/step - loss: 2.3024 - acc: 0.1071 - val_loss: 2.3022 - val_acc: 0.1214
Epoch 4/5
560/560 [==============================] - 345s 616ms/step - loss: 2.3026 - acc: 0.1071 - val_loss: 2.3028 - val_acc: 0.1214
Epoch 5/5
560/560 [==============================] - 335s 598ms/step - loss: 2.3024 - acc: 0.1071 - val_loss: 2.3024 - val_acc: 0.1214


In [0]:
#Third Model - Network In Network CNN

def network_cnn(model_input: Tensor) -> training.Model:
    
    #mlpconv block 1
    x = Conv2D(32, (5, 5), activation='relu',padding='valid')(model_input)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block2
    x = Conv2D(64, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = Conv2D(64, (1, 1), activation='relu')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.5)(x)
    
    #mlpconv block3
    x = Conv2D(128, (3, 3), activation='relu',padding='valid')(x)
    x = Conv2D(32, (1, 1), activation='relu')(x)
    x = Conv2D(10, (1, 1))(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Activation(activation='softmax')(x)
    
    model = Model(model_input, x, name='nin_cnn')
    
    return model
model_3 = network_cnn(mod_parameters)

In [0]:
#Network In Network CNN output
model3_history, model3_weights = build_model(model_3, number_of_epochs)

Train on 560 samples, validate on 140 samples
Epoch 1/5
560/560 [==============================] - 33s 59ms/step - loss: 2.3051 - acc: 0.0911 - val_loss: 2.3032 - val_acc: 0.1214
Epoch 2/5
560/560 [==============================] - 32s 57ms/step - loss: 2.3029 - acc: 0.1000 - val_loss: 2.3026 - val_acc: 0.0857
Epoch 3/5
560/560 [==============================] - 32s 58ms/step - loss: 2.3027 - acc: 0.1036 - val_loss: 2.3023 - val_acc: 0.1214
Epoch 4/5
560/560 [==============================] - 32s 57ms/step - loss: 2.3025 - acc: 0.1071 - val_loss: 2.3025 - val_acc: 0.1214
Epoch 5/5
560/560 [==============================] - 32s 58ms/step - loss: 2.3023 - acc: 0.1071 - val_loss: 2.3027 - val_acc: 0.1214


In [0]:
models = [model1, model_2,model_3]

In [0]:
#Ensemble model using all the three neural nets
def ensemble(models: List [training.Model], model_input: Tensor) -> training.Model:
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model

In [0]:
ensemble=ensemble(models,mod_parameters)

In [0]:
ensemble

In [0]:
def evaluate_error(model: training.Model) -> np.float64:
    pred = model.predict(X_val_parameters, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_val_category)) / y_val_category.shape[0]   
 
    return error
evaluate_error(model1)

10.0

In [0]:
evaluate_error(model_2)

10.0

In [0]:
evaluate_error(model_3)

10.0

In [0]:
evaluate_error(ensemble)

10.0